# Librerías generales

In [1]:
%%capture
%pip install PyPDF2
%pip install pypdf
%pip install pdfminer.six
%pip install langchain
%pip install zipfile
%pip install --upgrade --quiet llmsherpa
%pip install langchain_community
%pip install tiktoken
%pip install pandas
%pip install gensim
%pip install nltk

In [2]:
import pandas as pd
import PyPDF2 as pdf2
from ast import literal_eval
from pdfminer.high_level import extract_text
from gensim.models import Word2Vec
import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langchain.text_splitter import NLTKTextSplitter
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [23]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to C:\Users\Daniel
[nltk_data]     B\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Daniel
[nltk_data]     B\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Daniel
[nltk_data]     B\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


# Funciones

In [21]:
a,b = 'áéíóúüñÁÉÍÓÚÜÑ|,."_;:🎀🤷🏻‍♀️🤭🟢⚪️🙏🏻🔪💿💀🔥🚫💸😁😍💜💎💁🏽‍♀️🐟🍍✨👏🏻🌟💞💯🍺〽️❤️💕','aeiouunAEIOUUN                                                 '
trans = str.maketrans(a,b)

def cleanName(_):
  cleaned_text_list = []
  # try:
  _ = _.translate(trans).rstrip()
  # Convierte el texto a minúsculas
  _ = _.lower()

  # Elimina URLs
  _ = re.sub(r'http\S+|www\S+|https\S+', '', _, flags=re.MULTILINE)

  # Elimina las menciones @ y '#' de las redes sociales
  _ = re.sub(r'\@\w+|\#\w+', '', _)

  # Elimina los caracteres de puntuación
  _ = strip_punctuation(_)

  # Elimina los números
  _ = strip_numeric(_)

  # Elimina las palabras cortas
#   _ = strip_short(_,minsize=2)

  # Elimina las palabras comunes (stop words)
  stop_words = set(stopwords.words('spanish'))
  # stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(_)
  filtered_text = [word for word in word_tokens if word not in stop_words]
  cleaned_text_list.append(filtered_text)
  # print(cleaned_text_list)
  _ = ' '.join(cleaned_text_list[0]) #.apply(concatenar_lista)
  # except :
  #   _ = _

  return _ #{'text': _}

In [4]:
import os
# from google.colab import drive
# drive.mount('/content/drive')
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, os.pardir, "data")
os.path.exists(data_dir)

True

# Muestra PDT para clusters

In [5]:
import os
pdf_and_text = []
pages = None
# def readZip(dir):
bdl_corpus = pd.DataFrame()
# ruta = '/content/drive/MyDrive/Proyectos/NLP/ODS/Insumos/PDM-ABRIL-30.pdf'
ruta = os.path.join(data_dir,"pdf" ,"27.pdf")

loader = PyPDFLoader(f"{ruta}")
pages = loader.load_and_split()
pages_content=''

for item in pages:
  # print(type(item))
  pages_content = pages_content + item.page_content

pdf_and_text.append({"File Name":'Duitama',"text":pages_content.replace('\n', ' ')})

In [6]:
%pip install --upgrade --quiet langchain-text-splitters tiktoken

Note: you may need to restart the kernel to use updated packages.


In [12]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=100,
    chunk_overlap=0,
)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 128,
#     chunk_overlap  = 20,
#     length_function = len,
# )

# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     encoding_name="cl100k_base", chunk_size=256, chunk_overlap=20
# )
documents = text_splitter.split_text(pages_content.replace('\n', '').replace('\uf0b7', ''))

# documents = text_splitter.split_documents(pages)

In [24]:
documents = [cleanName(doc) for doc in documents]
documents

['republica colombia departamento choco nit republica colombia departamento choco nit tabla contenido pg presentacion mision vision principios',
 'premisas enfoques plan desarrollo departamental articulacion planes desarrollo capitulo i diagnostico generalidades departamento',
 'localizacion limites contexto demografico poblacion total estructura demografica',
 'dinamica demografica indicadores dinamica poblacion movilidad forzada dinamica migratoria',
 'linea estrategica choco incluyente corresponsable generar confianza sector salud abordaje efectos salud determinantes',
 'analisis mortalidad mortalidad general grandes causas mortalidad especifica subgrupo',
 'mortalidad materno – infantil ninez tasas especificas mortalidad ninez analisis desigualdad mortalidad materno – infantil ninez',
 'determinantes estructurales identificacion prioridades identificadas mortalidad general especifica materno –infantil analisis morbilidad',
 'principales causas morbilidad morbilidad especifica salud

In [25]:
# [txt['text'] for txt in pdf_and_text[0] if txt['File Name']=='Duitama']
print(len(documents[600].split(' ')))#.page_content
# [doc for doc in documents if f'|' in doc]
# len(documents)
# print(ascii('d'))
# type(pages)
# documents

36


In [8]:
%%capture
%pip install -U sentence-transformers
# %pip install torch

In [26]:
import torch
from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer("multi-qa-mpnet-base-cos-v1")
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
model = SentenceTransformer("hackathon-pln-es/paraphrase-spanish-distilroberta")


c:\Users\Daniel B\miniconda3\envs\nlp_base\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Daniel B\.cache\huggingface\hub\models--hackathon-pln-es--paraphrase-spanish-distilroberta. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Daniel B\miniconda3\envs\nlp_base\lib\site-packages\transfor

In [10]:
# Verifica si CUDA (GPU) está disponible
if torch.cuda.is_available():
    print("CUDA (GPU) está disponible.")
    print("Número de GPUs disponibles:", torch.cuda.device_count())
    print("Nombre de la GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA (GPU) no está disponible.")

CUDA (GPU) no está disponible.


In [27]:
passage_embeddings = model.encode(documents,batch_size=64,show_progress_bar=True)#, device='cuda')

Batches:  98%|█████████▊| 49/50 [14:27<00:17, 17.70s/it]


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [15]:
pd.DataFrame(passage_embeddings)#.to_csv(os.path.join(data_dir, "embeddings", "27.csv"), index=False)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.042367,-0.071221,0.048507,-0.045486,-0.002688,-0.057612,-0.004107,0.045282,0.022898,0.029578,...,-0.007496,-0.046049,0.038519,-0.069133,-0.080755,0.011094,0.052599,-0.009545,-0.067781,0.000760
1,0.015170,0.015056,-0.023920,-0.038653,-0.033825,-0.046896,-0.019105,0.082348,-0.044354,0.031231,...,0.000516,0.065708,-0.059714,-0.002351,-0.042863,0.014531,0.017540,-0.018974,-0.034688,-0.077253
2,0.127279,-0.054075,-0.044775,-0.070976,0.005485,-0.026359,0.043829,0.044072,-0.054648,0.001396,...,0.064478,0.046135,-0.010347,-0.007818,-0.050186,0.040959,-0.047093,0.005894,-0.007718,-0.011703
3,0.055317,0.042160,-0.046554,0.014982,-0.077174,-0.065201,0.031923,0.096568,-0.065272,0.014883,...,0.008923,0.020193,0.007194,0.037258,-0.091998,-0.070002,0.068594,-0.013958,-0.000622,-0.051965
4,-0.013688,-0.036379,-0.024030,-0.000450,-0.025156,0.028489,-0.045820,0.084136,-0.060630,-0.025134,...,-0.017208,-0.037420,0.043487,-0.020917,-0.063246,-0.015969,0.100551,0.061250,-0.003592,-0.046413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3146,0.085621,0.040211,0.028736,-0.041677,-0.008269,0.000217,-0.065977,0.122317,-0.018017,0.080615,...,0.054620,-0.013872,0.019714,-0.021273,-0.078078,-0.006569,0.016842,-0.022563,-0.024558,-0.024049
3147,0.054842,-0.009338,-0.014763,-0.032103,0.034911,-0.032972,0.037725,0.048795,0.012302,-0.000961,...,0.019112,0.017625,0.136299,0.046935,0.071095,0.041286,0.035396,0.068310,-0.084779,-0.048458
3148,0.081281,-0.005785,0.037971,-0.056628,-0.007179,-0.001117,-0.057305,0.127010,-0.015358,0.055377,...,0.013710,-0.074688,0.019592,0.001136,-0.060667,-0.017555,0.005003,-0.011887,-0.067822,-0.040591
3149,0.032226,0.087562,-0.025763,-0.024795,0.019876,0.041715,0.065621,0.079587,-0.052099,0.041287,...,-0.038746,-0.101054,0.116830,0.056482,-0.002732,0.045615,0.091708,0.022293,-0.068483,-0.087590


In [16]:
df_doc = pd.DataFrame(
    {"Documentos" : documents,
     'embeddings' : passage_embeddings.tolist()}
)
df_doc.head()
# passage_embeddings.shape

,Documentos,embeddings
0,1 REPUBLICA DE COLOMBIA DEPARTAMENTO DEL CHO...,"[0.04236729070544243, -0.0712207481265068, 0.0..."
1,PREMISAS 22 ENFOQUES DEL PLAN DE DESARRO...,"[0.015169757418334484, 0.015055638737976551, -..."
2,1.1.1 Localización y Limites 32 1.1.2...,"[0.12727926671504974, -0.05407460033893585, -0..."
3,1.1.2.3 Dinámica demográfica 40 1.1.2...,"[0.05531660094857216, 0.04216017946600914, -0...."
4,42 1.2 LÍNEA ESTRATÉGICA 1. UN CHOCÓ INCLUYEN...,"[-0.013687951490283012, -0.03637908026576042, ..."


In [15]:
# df_doc.to_csv('/content/drive/MyDrive/Proyectos/NLP/ODS/output/df_doc.txt', sep='|')

In [21]:
query_embedding = model.encode(['Proyecciones de la población'])
df_doc['similarity'] = df_doc['embeddings'].apply(lambda x: util.cos_sim(x,query_embedding))
df_doc.sort_values('similarity', ascending=False).head()

,Documentos,embeddings,similarity
210,proceso de prevención a los factores de riesgo...,"[-0.0302952378988266, 0.07318558543920517, -0....",[[tensor(0.7136)]]
391,"quintero, el cargua, rincón del cargua, las Am...","[-0.02997242659330368, 0.08857964724302292, -0...",[[tensor(0.6184)]]
223,ser uno de los pilares del accionar de todas l...,"[-0.011530615389347076, 0.07240030914545059, -...",[[tensor(0.6083)]]
232,por la ESE reflejan incremento de vigencia a v...,"[0.0012987136142328382, 0.014815008267760277, ...",[[tensor(0.5924)]]
326,proyectos nuevos. Esta adquisición de predios ...,"[-0.02225414291024208, 0.05412407964468002, -0...",[[tensor(0.5849)]]


In [23]:
df_doc.sort_values('similarity', ascending=False).head().reset_index(drop=True).iloc[1,0]

'quintero, el cargua, rincón del cargua, las Américas, Boyacá, Colombia, san Juan Bosco, los Alpes, Camilo torres, simón bolívar, barrio las lajas.;  De los delitos puestos en conocimiento la población femenina ha asumido 984 contra 831 que se le han efectuado a la población masculina. Para este nuevo periodo (2020'

# Ejemplo clusters

In [ ]:
%%capture
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Corpus with example sentences
corpus = [
    "A man is eating food.",
    "A man is eating a piece of bread.",
    "A man is eating pasta.",
    "The girl is carrying a baby.",
    "The baby is carried by the woman",
    "A man is riding a horse.",
    "A man is riding a white horse on an enclosed ground.",
    "A monkey is playing drums.",
    "Someone in a gorilla costume is playing a set of drums.",
    "A cheetah is running behind its prey.",
    "A cheetah chases prey on across a field.",
]
corpus_embeddings = embedder.encode(corpus)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
type(corpus_embeddings)#.shape()

numpy.ndarray

In [ ]:
# Some models don't automatically normalize the embeddings, in which case you should normalize the embeddings:
# corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform agglomerative clustering
clustering_model = AgglomerativeClustering(
    n_clusters=None, distance_threshold=1.5
)  # , affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i + 1)
    print(cluster)
    print("")

Cluster  1
['A man is eating food.', 'A man is eating a piece of bread.', 'A man is eating pasta.']

Cluster  5
['The girl is carrying a baby.', 'The baby is carried by the woman']

Cluster  2
['A man is riding a horse.', 'A man is riding a white horse on an enclosed ground.']

Cluster  3
['A monkey is playing drums.', 'Someone in a gorilla costume is playing a set of drums.']

Cluster  4
['A cheetah is running behind its prey.', 'A cheetah chases prey on across a field.']

